***
# Concatenating and merging data

Combine several `Series` or `DataFrame` objects:

1. [`pd.concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html): combine multiple Series/DataFrames by appending observations (rows) or columns.
2. [`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html): match observations from one Series/DataFrame with observations from another Series/DataFrame and combine these into a _merged_ DataFrame.

***
## Concatenation

### Concatenating Series

In [10]:
# Create demo series
import pandas as pd 

a = pd.Series(['A1', 'A2', 'A3'])
b = pd.Series([f'B{i}' for i in range(5)])

In [7]:
s = pd.concat((a,b), ignore_index=True, axis = 1)
s

,0,1
0,A1,B0
1,A2,B1
2,A3,B2
3,NaN,B3
4,NaN,B4


In [3]:
s.loc[0]

0    A1
0    B0
dtype: object

*Example: Concatenating two Series along the row axis*

-   Use [`pd.concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)
-   Optionally specify `axis=0` (the default)
-   Might need to reset index of result object
-   Index can be reset automatically by specifying `ignore_index=True`

*Example: Concatenating along the column axis*

-   Need to specify `axis=1` argument
-   Use `keys` to specify column names

<div class="alert alert-info">
<h3> Your turn</h3>
<ol>
    <li>Create a new <TT>Series</TT> with observations <TT>['C1', 'C2']</TT>.</li>
    <li>Using the previously created <TT>Series</TT> <TT>a</TT> and <TT>b</TT>, concatenate all three objects along the row axis and create a new (unique) index.</li>
    <li>Repeat the previous step, but now concatenate along the column axis. Assign the column names <TT>'Column1'</TT>, <TT>'Column2'</TT>, and <TT>'Column3'</TT>.</li>
</ol>
</div>

In [15]:
c = pd.Series(["C1", "C2"])

d = pd.concat((a,b,c), ignore_index=True, axis = 1, keys = ['Column1', 'Column2', 'Column3'])

d


,0,1,2
0,A1,B0,C1
1,A2,B1,C2
2,A3,B2,NaN
3,NaN,B3,NaN
4,NaN,B4,NaN


### Concatenating DataFrames

In [16]:
# Create demo DataFrames

import pandas as pd
import numpy as np

# Create 2 x 2 DataFrame of string data
df_a = pd.DataFrame(np.array([f'A{i}' for i in range(4)]).reshape((2, 2)))

# Create 2 x 3 DataFrame of string data
df_b = pd.DataFrame(np.array([f'B{i}' for i in range(6)]).reshape((2, 3)))

In [17]:
pd.concat((df_a, df_b), axis=1, keys = ['A', 'B'])

A       B        
    0   1   0   1   2
0  A0  A1  B0  B1  B2
1  A2  A3  B3  B4  B5

#### Concatenating along the column axis

In [20]:
pd.concat((df_a, df_b), axis=0, ignore_index=True)

,0,1,2
0,A0,A1,NaN
1,A2,A3,NaN
2,B0,B1,B2
3,B3,B4,B5


*Example: Concatenating two DataFrames along the column axis*

-   Might need to reset non-unique column index (or specify `ignore_index=True`)
-   Use `keys` argument to `concat()` to add outer level to column index

#### Concatenating along the row axis

*Example: Concatenating rows with identical columns*

*Example: Concatenating rows with different columns*

<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>FRED_monthly_1950.csv</TT> and <TT>FRED_monthly_1960.csv</TT> into two different DataFrames.
        The files contain monthly macroeconomic time series for the 1950s and 1960s, respectively.
        <p>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </p>
        </li>
    <li>Concatenate these DataFrames along the row dimension to get a total of 240 observations.</li>
    <li>Set the column <TT>DATE</TT> as index for the newly created DataFrame.</li>
</ol>
</div>

In [26]:
# Read in data used for mini-exercise

# Relative path to data directory
DATA_PATH = '../../data'

# Read in FRED data for 1950s and 1960s
df_1950 = pd.read_csv(f'{DATA_PATH}/FRED/FRED_monthly_1950.csv', parse_dates=['DATE'])
df_1960 = pd.read_csv(f'{DATA_PATH}/FRED/FRED_monthly_1960.csv', parse_dates=['DATE'])

df_total = pd.concat((df_1950, df_1960))
df_total.set_index("DATE", inplace=True)
df_total

,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART
DATE,,,,,
1950-01-01,23.5,6.5,NaN,NaN,58.9
1950-02-01,23.6,6.4,NaN,NaN,58.9
1950-03-01,23.6,6.3,NaN,NaN,58.8
1950-04-01,23.6,5.8,NaN,NaN,59.2
1950-05-01,23.8,5.5,NaN,NaN,59.1
...,...,...,...,...,...
1969-08-01,36.9,3.5,9.2,NaN,60.3
1969-09-01,37.1,3.7,9.2,NaN,60.3
1969-10-01,37.3,3.7,9.0,NaN,60.4


***
## Merging and joining data sets

### Types of merges

We can merge DataFrames `A` and `B` in various ways:

1.  *one-to-one*: Unique keys in `A` and `B`
2.  *many-to-one*: Unique keys in `A`, mapped to multiple keys in `B`
3.  *many-to-many*: Multiple keys in both `A` mapping to multiple keys in `B` (avoid this)

### Implementation in pandas

1.  [`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html): function that takes as argument the *two* DataFrames to be merged,
    e.g.,
    
    ```python
    result = pd.merge(df_A, df_B)
    ```
2.  [`df.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html): method of a specific `DataFrame` object, takes the other `DataFrame` as argument, e.g.,
    
    ```python
    result = df_A.merge(df_B)
    ```


### Controlling the resulting data set

Merge methods (controlled by `how` argument to `merge()`):

1.  `how='inner'` (_inner join_): _intersection_ of keys that are present in _both_ data sets
2.  `how='outer'` (_outer join_): _union_ of keys present in either of the data sets
3.  `how='left'` (_left join_): all identifiers from the _left_ data set are in the result
4.  `how='right'` (_right join_): all identifiers from the _right_ data set are in the result

Illustration: Each circle represents the keys present in the left (`df1`) or right (`df2`) DataFrames. The merge method controls which subset of keys is retained in the merge result.

![Join types](join-methods.png)

***
### Merging with `merge()`

#### One-to-one merges

In [27]:
# Create first DataFrame with 2 rows
df_a = pd.DataFrame({'key': [0, 1], 'value_a': ['A0', 'A1']})

# Create second DataFrame with 2 rows
df_b = pd.DataFrame({'key': [1, 2], 'value_b': ['B1', 'B2']})

*Example: inner join*

-   By default, `pd.merge()` performs inner join on all overlapping columns
-   Argument `on` controls on which columns to match (default: use all overlapping columns)
-   Argument `how` controls which rows to keep in result


In [32]:
pd.merge(df_a, df_b, how = "left", on="key")

,key,value_a,value_b
0,0,A0,NaN
1,1,A1,B1


*Example: outer, left, and right join*

-   Specify `how='outer'`, `how='left'`, `how='right'`

<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>CPI.csv</TT> and <TT>GDP.csv</TT> into two different DataFrames.
        The files contain monthly data for the Consumer Price Index (CPI) and quarterly data for GDP, respectively.
        <p>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </p>
    </li>
    <li>Merge the CPI and the GDP time series with 
    <a href="https://pandas.pydata.org/docs/reference/api/pandas.merge.html"><TT>pd.merge()</TT></a> 
    without specifying the <TT>how=</TT> argument.</li>
    <li>Merge the CPI and the GDP time series with <TT>pd.merge()</TT>
    using a left join (<TT>how='left'</TT>).</li>
    <li>Merge the CPI and the GDP time series with <TT>pd.merge()</TT> using an inner join (<TT>how='inner'</TT>).</li>
</ol>
How do the results differ depending on the value of the <TT>how</TT> argument?
</div>

In [42]:
# Data used for mini-exercise
df_cpi = pd.read_csv(f'{DATA_PATH}/FRED/CPI.csv', parse_dates=['DATE'])
df_gdp = pd.read_csv(f'{DATA_PATH}/FRED/GDP.csv', parse_dates=['DATE'])

print(len(df_gdp["GDP"]))

pd.merge(df_cpi,df_gdp, on="DATE", how="inner")#.set_index("DATE", inplace=True)


312


,DATE,CPI,GDP
0,1947-01-01,21.5,2182.7
1,1947-04-01,22.0,2176.9
2,1947-07-01,22.2,2172.4
3,1947-10-01,22.9,2206.5
4,1948-01-01,23.7,2239.7
...,...,...,...
307,2023-10-01,307.7,22960.6
308,2024-01-01,309.8,23053.5
309,2024-04-01,313.0,23223.9
310,2024-07-01,313.6,23400.3


#### Many-to-one merges

*Example: merge unique values in `df_b` to multiple values in `df_a`*

In [43]:
# Create first DataFrame with 4 rows, non-unique key
df_a = pd.DataFrame({'key': [0, 1, 0, 1], 'value_a': ['A0', 'A1', 'A2', 'A3']})

# Create second DataFrame with 2 rows, unique key
df_b = pd.DataFrame({'key': [0, 1], 'value_b': ['B0', 'B1']})

In [44]:
df_a

,key,value_a
0,0,A0
1,1,A1
2,0,A2
3,1,A3


In [45]:
df_b

,key,value_b
0,0,B0
1,1,B1


In [47]:
pd.merge(df_a, df_b, validate="m:1")

,key,value_a,value_b
0,0,A0,B0
1,1,A1,B1
2,0,A2,B0
3,1,A3,B1


***
### Merging with `DataFrame.merge()`

-   Equivalent to `pd.merge()`

*Example: repeat one-to-one merges from above*

In [50]:
# Create first DataFrame with 2 rows
df_a = pd.DataFrame({'key': [0, 1], 'value_a': ['A0', 'A1']})

# Create second DataFrame with 2 rows
df_b = pd.DataFrame({'key': [1, 2], 'value_b': ['B1', 'B2']})

*Example: Merging with overlapping column names*

In [51]:
# Rename columns to common name 'value'
df_a = df_a.rename(columns={'value_a': 'value'})
df_b = df_b.rename(columns={'value_b': 'value'})

In [53]:
df_a.merge(df_b, on="key", how="outer")

,key,value_x,value_y
0,0,A0,NaN
1,1,A1,B1
2,2,NaN,B2


<div class="alert alert-info">
<h3> Your turn</h3>
Repeat the previous exercise, but use the <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html"><TT>DataFrame.merge()</TT></a> 
method to perform the merge.
<ol>
    <li>Load the data in <TT>CPI.csv</TT> and <TT>GDP.csv</TT> into two different DataFrames.
        The files contain monthly data for the Consumer Price Index (CPI) and quarterly data for GDP, respectively.
        <p>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </p>
    </li>
    <li>Merge the DataFrame containing the CPI with the DataFrame containing GDP without specifying the <TT>how=</TT> argument.</li>
    <li>Merge the DataFrame containing the CPI with the DataFrame containing GDP using a <i>left</i> join.</li>
    <li>Merge the DataFrame containing the CPI with the DataFrame containing GDP using an <i>inner</i> join.</li>
</ol>
How do the results differ depending on the value of the <TT>how</TT> argument?
</div>

In [ ]:
# Data used for mini-exercise
df_cpi = pd.read_csv(f'{DATA_PATH}/FRED/CPI.csv', parse_dates=['DATE'])
df_gdp = pd.read_csv(f'{DATA_PATH}/FRED/GDP.csv', parse_dates=['DATE'])

***
### Joining with `join()`

The `DataFrame` method 
[`join()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html) 
is a convenience wrapper around 
[`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html):

1.  `join()` can be called _only_ directly on the `DataFrame` object
2.  `join()` always operates on the _index_ of the other `DataFrame`
3.  `join()` by default performs a `left` join, whereas `merge()` performs an `inner` join

Use `join()` if you want to join DataFrames which have a similar index.

*Example: joining DataFrames*

In [ ]:
# Create first DataFrame with 2 rows
df_a = pd.DataFrame(['A0', 'A1'], columns=['value_a'], index=[0, 1])

# Create second DataFrame with 2 rows
df_b = pd.DataFrame(['B1', 'B2'], columns=['value_b'], index=[1, 2])

<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>CPI.csv</TT> and <TT>GDP.csv</TT> into two different DataFrames.
        The files contain monthly data for the Consumer Price Index (CPI) and quarterly data for GDP, respectively.
        <br/>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </li>
    <li>Set the <TT>DATE</TT> column as the index for each of the two DataFrames.</li>
    <li>Merge the CPI with the GDP time series with 
    <a href="https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html"><TT>join()</TT></a>. 
    Do this with both a left and an inner join.</li>
</ol>
</div>

In [ ]:
# Data used for mini-exercise
df_cpi = pd.read_csv(f'{DATA_PATH}/FRED/CPI.csv', parse_dates=['DATE'])
df_gdp = pd.read_csv(f'{DATA_PATH}/FRED/GDP.csv', parse_dates=['DATE'])

***
# Dealing with missing values

-   Often created as result of `pd.merge()` or `pd.concat()` if operands don't have the same row or column index

In [58]:
# Create two DataFrames with partially overlapping keys
df_a = pd.DataFrame({'key': [0, 1], 'value_a': ['A0', 'A1']})
df_b = pd.DataFrame({'key': [1, 2], 'value_b': ['B1', 'B2']})

# Merge with outer join, thus creating missing values
df = pd.merge(df_a, df_b, on='key', how='outer')
df

,key,value_a,value_b
0,0,A0,NaN
1,1,A1,B1
2,2,NaN,B2


## Dropping missing values

Missing values can be dropped by either

1.  Using [`dropna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)
2.  Selecting a subset of observations with a boolean operation such as 
    [`notna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.notna.html) or 
    [`isna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html).
2. Avoiding the missing values in the first place, e.g., by using `merge(..., how='inner')`.

*Example: Dropping missing values*

- Use _outer/left/right join_ and drop missing data with `dropna()`

<div class="alert alert-info">
<h3> Your turn</h3>
Instead of using <TT>dropna()</TT>, drop the missing observations using either <TT>notna()</TT> or <TT>isna()</TT>.
<ol>
    <li>Drop all rows with any missing observations.</li>
    <li>Drop all rows with missing observations in column <TT>value_a</TT>.</li>
</ol>
</div>

In [60]:
df.loc[df["value_a"].notna()]

,key,value_a,value_b
0,0,A0,NaN
1,1,A1,B1


*Example: Avoiding missing values in the first place*

- E.g., use _inner join_ when merging data sets

## Filling missing values

Instead of dropping data, we can impute missing values in various ways:

1.  [`fillna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html): replace missing data with user-specified values
2.  [`ffill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html) and 
    [`bfill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bfill.html): fill missing values
    forward or backward from adjacent non-missing observations
3.  [`interpolate()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html): interpolate missing values based on non-missing ones

*Example: Replacing missing values with `fillna()`*

- Specify one value for _entire_ `DataFrame`
- Specify different value for each column using a dictionary

*Example: forward- or backward-filling missing values*

- Use [`ffill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html) to forward-fill
- Use [`bfill()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bfill.html) to backward-fill

*Example: linear interpolation*

- Use [`interpolate()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.interpolate.html) to interpolate missing _numerical_ data (e.g., `method='linear'`)

In [ ]:
s = pd.Series([1.0, 2.0, 3.0, np.nan, 5.0])

<div class="alert alert-info">
<h3> Your turn</h3>
Use the data files located in the folder <TT>../../data/FRED</TT> to perform the following tasks:
<ol>
    <li>Load the data in <TT>CPI.csv</TT> and <TT>GDP.csv</TT> into two different DataFrames.
        The files contain monthly data for the Consumer Price Index (CPI) and quarterly data for GDP, respectively.
        <br/>
        <i>Hint:</i> Use <TT>pd.read_csv(..., parse_dates=['DATE'])</TT> to automatically parse strings stored in the <TT>DATE</TT> column as dates.
        </li>
    <li>Merge the CPI with the GDP time series with <TT>merge()</TT> using a left join. This creates missing values in the <TT>GDP</TT>
    column.</li>
    <li>Impute the missing GDP values using <a href="https://pandas.pydata.org/docs/reference/api/pandas.Series.interpolate.html"><TT>interpolate()</TT></a> 
    and replace the missing values in column <TT>GDP</TT>.</li>
</ol>
</div>